In [4]:
import numpy as np 

In [5]:
y = np.array([[0.],
       [1.],
       [1.]])


MAX_ITERS = 2
GAMMA = 0.1


tx = np.array([[2.3, 3.2],
       [1. , 0.1],
       [1.4, 2.3]])

initial_w = np.array([[0.5],
       [1. ]])

In [6]:

def compute_logistic_reg_loss(y, tx, w, lambda_): 

    pred = np.dot(tx, w)
    sigmoids = 1.0 / (1 + np.exp(-pred))
    loss = -np.mean(y * np.log(sigmoids) + (1 - y) * np.log(1 - sigmoids)) + (lambda_ / 2) * np.linalg.norm(w)**2
    return sigmoids, loss

def reg_logistic_regression(y, tx, initial_w, gamma, max_iters, lambda_):
    """
    Perform regularized logistic regression using gradient descent.
    
    Parameters:
    y: np.array
        The target values
    tx: np.array
        The data matrix (each row is a data point)
    initial_w: np.array
        Initial weights
    max_iters: int
        Maximum number of iterations for gradient descent
    gamma: float
        Learning rate
    lambda_: float
        Regularization strength for L2 penalty

    Returns:
    w: np.array
        Optimized weights after training
    """
    
    w = initial_w

    sigmoids, loss = compute_logistic_reg_loss(y, tx, w, lambda_)

    
    for iter in range(max_iters):

        # compute the gradient
        grad = tx.T.dot(sigmoids - y)/len(y) + 2 * lambda_ * w

        # update w through the negative gradient direction
        w = w - gamma * grad

        sigmoids, loss = compute_logistic_reg_loss(y, tx, w, lambda_)
        
    return w, loss

In [7]:
reg_logistic_regression(y, tx, initial_w, MAX_ITERS , GAMMA, 0.4)


TypeError: reg_logistic_regression() missing 1 required positional argument: 'lambda_'